<a href="https://colab.research.google.com/github/juniismai02/kanker-kulit-vggnet-mobilenet-juniismail/blob/main/kanker_kulit_vggnet_mobilenet_juniismail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow numpy pandas matplotlib seaborn scikit-learn pillow tqdm kaggle albumentations opencv-python tensorflow-addons wandb psutil

In [3]:
import tensorflow as tf

# Cek versi dan GPU
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Konfigurasi memory growth
try:
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("Memory growth enabled on GPU(s)")
    else:
        print("No GPU detected, running on CPU")
except:
    print("Error configuring GPU")

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth enabled on GPU(s)


In [4]:
# Cek status GPU
!nvidia-smi

# Cek devices yang tersedia di TensorFlow
import tensorflow as tf
print("\nTensorFlow dapat melihat device berikut:")
print(tf.config.list_physical_devices())

Tue Feb 11 17:37:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 3. Buat struktur direktori
!mkdir -p "/content/kanker-kulit-projek/src"
!mkdir -p "/content/kanker-kulit-projek/data/raw"
!mkdir -p "/content/kanker-kulit-projek/models/checkpoints"
!mkdir -p "/content/kanker-kulit-projek/results/figures"
!mkdir -p "/content/kanker-kulit-projek/logs"

In [7]:
# 4. Setup Kaggle
from google.colab import files
files.upload()  # Upload kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [8]:
# 5. Download dataset
!kaggle datasets download kmader/skin-cancer-mnist-ham10000
!unzip skin-cancer-mnist-ham10000.zip -d "/content/kanker-kulit-projek/data/raw/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: /content/kanker-kulit-projek/data/raw/ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: /conte

In [32]:
with open('/content/kanker-kulit-projek/src/config.py', 'w') as f:
    f.write('''import os
import tensorflow as tf
import numpy as np
from datetime import datetime

class Config:
    """Konfigurasi untuk model klasifikasi kanker kulit"""

    # Parameter dasar
    RANDOM_SEED = 42
    IMG_SIZE = 224
    BATCH_SIZE = 16  # Optimized batch size
    NUM_CLASSES = 3
    EPOCHS = 200     # Increased epochs for better convergence

    # Pembagian dataset
    TRAIN_SPLIT = 0.7
    VAL_SPLIT = 0.15
    TEST_SPLIT = 0.15

    # Path direktori
    BASE_DIR = "/content/kanker-kulit-projek"
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
    PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
    MODEL_DIR = os.path.join(BASE_DIR, 'models')
    CHECKPOINT_DIR = os.path.join(MODEL_DIR, 'checkpoints')
    RESULTS_DIR = os.path.join(BASE_DIR, 'results')

    # Kelas sesuai dataset
    CLASS_MAPPING = {
        'bcc': 'Karsinoma Sel Basal',
        'mel': 'Melanoma',
        'akiec': 'Keratosis Aktinik'
    }

    # Parameter model yang dioptimalkan
    LEARNING_RATE = 1e-4
    MIN_LEARNING_RATE = 1e-6
    DROPOUT_RATE = 0.4     # Adjusted dropout rate
    L2_LAMBDA = 1e-4       # L2 regularization

    # Parameter augmentasi data yang ditingkatkan
    AUGMENTATION_PARAMS = {
        'rotation_range': 30,
        'width_shift_range': 0.2,
        'height_shift_range': 0.2,
        'shear_range': 15,
        'zoom_range': 0.2,
        'horizontal_flip': True,
        'vertical_flip': True,
        'fill_mode': 'reflect',
        'brightness_range': [0.8, 1.2],
        'channel_shift_range': 30,
        'random_contrast_range': [0.8, 1.2],
        'random_saturation_range': [0.8, 1.2],
        'random_hue_delta': 0.1,
        'mixup_alpha': 0.2,
        'cutmix_alpha': 1.0,
        'random_crop_size': (224, 224),
        'gaussian_noise_std': 0.01
    }

    # Parameter training yang dioptimalkan
    EARLY_STOPPING_PATIENCE = 25
    REDUCE_LR_PATIENCE = 10
    REDUCE_LR_FACTOR = 0.5
    WARMUP_EPOCHS = 5
    GRAD_CLIP_NORM = 1.0
    LABEL_SMOOTHING = 0.1

    # Parameter arsitektur
    ATTENTION_RATIO = 16
    FPN_FEATURE_SIZE = 256
    BACKBONE_FREEZE_LAYERS = {
        'vgg16': -8,        # Freeze semua layer kecuali 8 terakhir
        'mobilenet': -30,   # Freeze semua layer kecuali 30 terakhir
        'efficientnet': -20 # Freeze semua layer kecuali 20 terakhir
    }

    def __init__(self):
        """Initialize configuration"""
        self.timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        self._create_directories()
        self._set_random_seed()
        self._validate_config()
        self._log_config()

    def _create_directories(self):
        """Create necessary directories with timestamp"""
        directories = [
            self.PROCESSED_DATA_DIR,
            self.CHECKPOINT_DIR,
            self.RESULTS_DIR,
            os.path.join(self.RESULTS_DIR, 'plots'),
            os.path.join(self.RESULTS_DIR, 'logs'),
            os.path.join(self.RESULTS_DIR, f'experiment_{self.timestamp}')
        ]

        for dir_path in directories:
            os.makedirs(dir_path, exist_ok=True)

    def _set_random_seed(self):
        """Set random seeds for reproducibility"""
        os.environ['PYTHONHASHSEED'] = str(self.RANDOM_SEED)
        tf.random.set_seed(self.RANDOM_SEED)
        np.random.seed(self.RANDOM_SEED)

    def _validate_config(self):
        """Validate configuration parameters"""
        # Validate splits
        total_split = self.TRAIN_SPLIT + self.VAL_SPLIT + self.TEST_SPLIT
        if not np.isclose(total_split, 1.0):
            raise ValueError(f"Split fractions must sum to 1.0, got {total_split}")

        # Validate class mapping
        if len(self.CLASS_MAPPING) != self.NUM_CLASSES:
            raise ValueError(f"Number of classes ({self.NUM_CLASSES}) doesn't match CLASS_MAPPING ({len(self.CLASS_MAPPING)})")

        # Validate learning rates
        if self.MIN_LEARNING_RATE >= self.LEARNING_RATE:
            raise ValueError("MIN_LEARNING_RATE should be less than LEARNING_RATE")

        # Validate image size
        if self.IMG_SIZE % 32 != 0:
            raise ValueError("IMG_SIZE should be divisible by 32 for optimal performance")

    def _log_config(self):
        """Log configuration parameters"""
        config_path = os.path.join(
            self.RESULTS_DIR,
            f'experiment_{self.timestamp}',
            'config.txt'
        )

        with open(config_path, 'w') as f:
            f.write("KONFIGURASI EKSPERIMEN\\n")
            f.write(f"Timestamp: {self.timestamp}\\n\\n")

            f.write("Parameter Dasar:\\n")
            f.write(f"- Image Size: {self.IMG_SIZE}\\n")
            f.write(f"- Batch Size: {self.BATCH_SIZE}\\n")
            f.write(f"- Epochs: {self.EPOCHS}\\n")
            f.write(f"- Number of Classes: {self.NUM_CLASSES}\\n\\n")

            f.write("Learning Parameters:\\n")
            f.write(f"- Learning Rate: {self.LEARNING_RATE}\\n")
            f.write(f"- Minimum Learning Rate: {self.MIN_LEARNING_RATE}\\n")
            f.write(f"- Dropout Rate: {self.DROPOUT_RATE}\\n")
            f.write(f"- L2 Lambda: {self.L2_LAMBDA}\\n\\n")

            f.write("Model Architecture:\\n")
            f.write(f"- Attention Ratio: {self.ATTENTION_RATIO}\\n")
            f.write(f"- FPN Feature Size: {self.FPN_FEATURE_SIZE}\\n")
            f.write("- Backbone Freeze Layers:\\n")
            for backbone, layers in self.BACKBONE_FREEZE_LAYERS.items():
                f.write(f"  * {backbone}: {layers}\\n")

    def get_experiment_dir(self):
        """Get current experiment directory"""
        return os.path.join(self.RESULTS_DIR, f'experiment_{self.timestamp}')

    def get_model_params(self):
        """Get model-specific parameters"""
        return {
            'img_size': self.IMG_SIZE,
            'num_classes': self.NUM_CLASSES,
            'dropout_rate': self.DROPOUT_RATE,
            'l2_lambda': self.L2_LAMBDA,
            'attention_ratio': self.ATTENTION_RATIO,
            'fpn_feature_size': self.FPN_FEATURE_SIZE,
            'backbone_freeze_layers': self.BACKBONE_FREEZE_LAYERS
        }

    def get_training_params(self):
        """Get training-specific parameters"""
        return {
            'batch_size': self.BATCH_SIZE,
            'epochs': self.EPOCHS,
            'learning_rate': self.LEARNING_RATE,
            'min_learning_rate': self.MIN_LEARNING_RATE,
            'early_stopping_patience': self.EARLY_STOPPING_PATIENCE,
            'reduce_lr_patience': self.REDUCE_LR_PATIENCE,
            'reduce_lr_factor': self.REDUCE_LR_FACTOR,
            'warmup_epochs': self.WARMUP_EPOCHS,
            'grad_clip_norm': self.GRAD_CLIP_NORM,
            'label_smoothing': self.LABEL_SMOOTHING
        }
''')

In [28]:
with open('/content/kanker-kulit-projek/src/data_processor.py', 'w') as f:
    f.write('''import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Menekan warning CUDA
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

class DataProcessor:  # Mengubah nama kelas menjadi DataProcessor
    def __init__(self, config):
        self.config = config
        self.augmentation = self._create_augmentation_pipeline()

    def _create_augmentation_pipeline(self):
        """Membuat pipeline augmentasi yang kuat"""
        return A.Compose([
            # Transformasi geometrik dasar
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Transpose(p=0.5),

            # Noise dan blur
            A.OneOf([
                A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
            ], p=0.2),

            A.OneOf([
                A.MotionBlur(blur_limit=3, p=0.2),
                A.MedianBlur(blur_limit=3, p=0.1),
                A.Blur(blur_limit=3, p=0.1),
            ], p=0.2),

            # Transformasi geometrik kompleks
            A.ShiftScaleRotate(
                shift_limit=0.0625,
                scale_limit=0.2,
                rotate_limit=45,
                p=0.2
            ),

            # Distorsi
            A.OneOf([
                A.OpticalDistortion(distort_limit=1.0, p=0.3),
                A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
                A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.3),
            ], p=0.2),

            # Penyesuaian warna dan kontras
            A.OneOf([
                A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.3),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
            ], p=0.3),

            A.HueSaturationValue(
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=0.3
            ),

            # Normalisasi
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ])

    def prepare_data(self):
        """Method utama untuk mempersiapkan data"""
        try:
            # Load metadata
            df = self.load_metadata()

            # Split data
            train_df, val_df, test_df = self.split_data(df)

            # Setup generators
            train_generator, val_generator, test_generator = self.setup_data_generators(
                train_df, val_df, test_df
            )

            return train_generator, val_generator, test_generator

        except Exception as e:
            print(f"Error dalam prepare_data: {str(e)}")
            raise e

    def verify_paths(self):
        """Verifikasi keberadaan path yang diperlukan"""
        required_paths = [
            os.path.join(self.config.RAW_DATA_DIR, "HAM10000_metadata.csv"),
            os.path.join(self.config.RAW_DATA_DIR, "HAM10000_images_part_1"),
            os.path.join(self.config.RAW_DATA_DIR, "HAM10000_images_part_2")
        ]

        missing_paths = []
        for path in required_paths:
            if not os.path.exists(path):
                missing_paths.append(path)
                print(f"Warning: Path tidak ditemukan: {path}")

        return len(missing_paths) == 0

    def load_metadata(self):
        """Load dan preprocessing metadata"""
        if not self.verify_paths():
            raise ValueError("Beberapa path yang diperlukan tidak ditemukan!")

        try:
            metadata_path = os.path.join(self.config.RAW_DATA_DIR, "HAM10000_metadata.csv")
            df = pd.read_csv(metadata_path)
            df = df[df["dx"].isin(self.config.CLASS_MAPPING.keys())]

            min_class_count = df["dx"].value_counts().min()
            balanced_dfs = []

            for class_name in self.config.CLASS_MAPPING.keys():
                class_df = df[df["dx"] == class_name]
                if len(class_df) > min_class_count:
                    class_df = class_df.sample(n=min_class_count, random_state=self.config.RANDOM_SEED)
                elif len(class_df) < min_class_count:
                    augmented_samples = []
                    samples_needed = min_class_count - len(class_df)

                    for _ in range(samples_needed):
                        sample = class_df.sample(n=1, random_state=np.random.randint(0, 10000))
                        augmented_samples.append(sample)

                    augmented_df = pd.concat(augmented_samples, ignore_index=True)
                    class_df = pd.concat([class_df, augmented_df], ignore_index=True)

                balanced_dfs.append(class_df)

            balanced_df = pd.concat(balanced_dfs, ignore_index=True)
            balanced_df["label"] = pd.Categorical(balanced_df["dx"]).codes
            balanced_df["image_path"] = balanced_df["image_id"].apply(self._get_valid_image_path)
            balanced_df = balanced_df.dropna(subset=["image_path"])

            print(f"\\nTotal data setelah balancing: {len(balanced_df)}")
            print("\\nDistribusi kelas setelah balancing:")
            print(balanced_df["dx"].value_counts())

            return balanced_df

        except Exception as e:
            print(f"Error dalam load_metadata: {str(e)}")
            raise e

    def _get_valid_image_path(self, image_id):
        """Mendapatkan path gambar yang valid"""
        try:
            possible_paths = [
                os.path.join(
                    self.config.RAW_DATA_DIR,
                    "HAM10000_images_part_1",
                    f"{image_id}.jpg"
                ),
                os.path.join(
                    self.config.RAW_DATA_DIR,
                    "HAM10000_images_part_2",
                    f"{image_id}.jpg"
                )
            ]

            for path in possible_paths:
                if os.path.exists(path):
                    return path
            return None

        except Exception as e:
            print(f"Error dalam _get_valid_image_path: {str(e)}")
            return None

    def create_data_generators(self):
        """Membuat data generator dengan augmentasi yang ditingkatkan"""
        try:
            train_datagen = ImageDataGenerator(
                preprocessing_function=self._preprocess_input,
                rotation_range=45,
                width_shift_range=0.3,
                height_shift_range=0.3,
                shear_range=20,
                zoom_range=0.3,
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode="reflect",
                brightness_range=[0.7, 1.3],
                channel_shift_range=50
            )

            val_test_datagen = ImageDataGenerator(
                preprocessing_function=self._preprocess_input
            )

            return train_datagen, val_test_datagen

        except Exception as e:
            print(f"Error dalam create_data_generators: {str(e)}")
            raise e

    def _preprocess_input(self, x):
        """Preprocessing input images"""
        try:
            x = x.astype(np.float32) / 255.0
            mean = np.mean(x, axis=(0,1))
            std = np.std(x, axis=(0,1))
            x = (x - mean) / (std + 1e-7)
            return x

        except Exception as e:
            print(f"Error dalam _preprocess_input: {str(e)}")
            return x

    def split_data(self, df):
        """Split data dengan stratifikasi"""
        try:
            if len(df) == 0:
                raise ValueError("Dataset kosong! Periksa path gambar!")

            train_val_df, test_df = train_test_split(
                df,
                test_size=self.config.TEST_SPLIT,
                stratify=df["dx"],
                random_state=self.config.RANDOM_SEED
            )

            train_df, val_df = train_test_split(
                train_val_df,
                test_size=self.config.VAL_SPLIT/(1-self.config.TEST_SPLIT),
                stratify=train_val_df["dx"],
                random_state=self.config.RANDOM_SEED
            )

            print(f"Jumlah data training: {len(train_df)}")
            print(f"Jumlah data validasi: {len(val_df)}")
            print(f"Jumlah data testing: {len(test_df)}")

            return train_df, val_df, test_df

        except Exception as e:
            print(f"Error dalam split_data: {str(e)}")
            raise e

    def setup_data_generators(self, train_df, val_df, test_df):
        """Setup data generators dengan augmentasi"""
        try:
            train_datagen, val_test_datagen = self.create_data_generators()

            train_generator = self._create_generator(train_datagen, train_df, shuffle=True)
            val_generator = self._create_generator(val_test_datagen, val_df, shuffle=False)
            test_generator = self._create_generator(val_test_datagen, test_df, shuffle=False)

            return train_generator, val_generator, test_generator

        except Exception as e:
            print(f"Error dalam setup_data_generators: {str(e)}")
            raise e

    def _create_generator(self, datagen, df, shuffle):
        """Helper function untuk membuat generator"""
        try:
            return datagen.flow_from_dataframe(
                df,
                x_col="image_path",
                y_col="dx",
                target_size=(self.config.IMG_SIZE, self.config.IMG_SIZE),
                batch_size=self.config.BATCH_SIZE,
                class_mode="categorical",
                classes=list(self.config.CLASS_MAPPING.keys()),
                shuffle=shuffle,
                validate_filenames=True
            )

        except Exception as e:
            print(f"Error dalam _create_generator: {str(e)}")
            raise e

    def create_k_fold_splits(self, df, n_splits=5):
        """Membuat K-fold cross validation splits"""
        try:
            kf = StratifiedKFold(
                n_splits=n_splits,
                shuffle=True,
                random_state=self.config.RANDOM_SEED
            )

            splits = []
            for train_idx, val_idx in kf.split(df, df["dx"]):
                train_fold = df.iloc[train_idx]
                val_fold = df.iloc[val_idx]
                splits.append((train_fold, val_fold))

            return splits

        except Exception as e:
            print(f"Error dalam create_k_fold_splits: {str(e)}")
            raise e
''')

In [36]:
with open('/content/kanker-kulit-projek/src/model.py', 'w') as f:
    f.write('''import tensorflow as tf
from tensorflow.keras.applications import VGG16, MobileNetV2, EfficientNetB0
from tensorflow.keras.layers import (
    Dense, GlobalAveragePooling2D, Concatenate, Dropout, Input,
    BatchNormalization, Add, Multiply, Activation, Reshape, Layer,
    Conv2D, UpSampling2D
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

class SpatialAttention(Layer):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.kernel_size = kernel_size

    def build(self, input_shape):
        self.conv = Conv2D(
            filters=1,
            kernel_size=self.kernel_size,
            padding='same',
            kernel_initializer='he_normal'
        )

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        attention = self.conv(concat)
        attention = tf.sigmoid(attention)
        return tf.multiply(inputs, attention)

class ChannelAttention(Layer):
    def __init__(self, ratio=16):
        super(ChannelAttention, self).__init__()
        self.ratio = ratio

    def build(self, input_shape):
        channel = input_shape[-1]
        self.shared_layer_one = Dense(channel // self.ratio,
                                    activation='relu',
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')
        self.shared_layer_two = Dense(channel,
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')

    def call(self, inputs):
        avg_pool = GlobalAveragePooling2D()(inputs)
        max_pool = GlobalAveragePooling2D()(inputs)

        avg_pool = Reshape((1, 1, inputs.shape[-1]))(avg_pool)
        max_pool = Reshape((1, 1, inputs.shape[-1]))(max_pool)

        avg_out = self.shared_layer_one(avg_pool)
        max_out = self.shared_layer_one(max_pool)

        avg_out = self.shared_layer_two(avg_out)
        max_out = self.shared_layer_two(max_out)

        outputs = tf.add(avg_out, max_out)
        outputs = tf.sigmoid(outputs)

        return tf.multiply(inputs, outputs)

class CBAM(Layer):
    def __init__(self):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention()
        self.spatial_attention = SpatialAttention()

    def call(self, inputs):
        x = self.channel_attention(inputs)
        x = self.spatial_attention(x)
        return x

class ModelBuilder:
    def __init__(self, config):
        self.config = config
        self.model = None

    def build(self):
        """Build model architecture"""
        # Input layer
        inputs = Input(shape=(self.config.IMG_SIZE, self.config.IMG_SIZE, 3))

        # Base models
        vgg = VGG16(
            include_top=False,
            weights='imagenet',
            input_tensor=inputs
        )

        mobilenet = MobileNetV2(
            include_top=False,
            weights='imagenet',
            input_tensor=inputs
        )

        efficientnet = EfficientNetB0(
            include_top=False,
            weights='imagenet',
            input_tensor=inputs
        )

        # Freeze layers
        for layer in vgg.layers[:-8]:
            layer.trainable = False
        for layer in mobilenet.layers[:-30]:
            layer.trainable = False
        for layer in efficientnet.layers[:-20]:
            layer.trainable = False

        # Extract features
        vgg_features = vgg.output
        mobilenet_features = mobilenet.output
        efficientnet_features = efficientnet.output

        # Apply CBAM attention
        vgg_att = CBAM()(vgg_features)
        mobilenet_att = CBAM()(mobilenet_features)
        efficientnet_att = CBAM()(efficientnet_features)

        # Global pooling
        vgg_pool = GlobalAveragePooling2D()(vgg_att)
        mobilenet_pool = GlobalAveragePooling2D()(mobilenet_att)
        efficientnet_pool = GlobalAveragePooling2D()(efficientnet_att)

        # Concatenate features
        merged = Concatenate()([vgg_pool, mobilenet_pool, efficientnet_pool])

        # Dense layers with residual connections
        x = Dense(1024, kernel_regularizer=l2(self.config.L2_LAMBDA))(merged)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(self.config.DROPOUT_RATE)(x)

        # First residual block
        residual = x
        x = Dense(512, kernel_regularizer=l2(self.config.L2_LAMBDA))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(self.config.DROPOUT_RATE)(x)
        x = Dense(512, kernel_regularizer=l2(self.config.L2_LAMBDA))(x)
        x = BatchNormalization()(x)
        residual = Dense(512)(residual)  # Match dimensions
        x = Add()([x, residual])
        x = Activation('relu')(x)

        # Output layer
        outputs = Dense(
            self.config.NUM_CLASSES,
            activation='softmax',
            kernel_regularizer=l2(self.config.L2_LAMBDA)
        )(x)

        # Create model
        self.model = Model(inputs=inputs, outputs=outputs, name='skin_cancer_model')

        # Compile model
        self._compile_model()

        return self.model

    def _compile_model(self):
        """Compile model with optimized settings"""
        initial_learning_rate = self.config.LEARNING_RATE

        lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
            initial_learning_rate,
            first_decay_steps=1000,
            t_mul=2.0,
            m_mul=0.9,
            alpha=0.1
        )

        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule,
            clipnorm=self.config.GRAD_CLIP_NORM
        )

        self.model.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=self.config.LABEL_SMOOTHING),
            metrics=[
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.AUC(name='auc')
            ]
        )
''')

In [38]:
with open('/content/kanker-kulit-projek/src/utils.py', 'w') as f:
    f.write('''import tensorflow as tf
import os
import time
import cv2
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve

class Utils:
    """Utility functions untuk klasifikasi kanker kulit"""

    @staticmethod
    def setup_gpu():
        """Setup GPU configuration"""
        try:
            # Reset session untuk menghindari konflik
            tf.keras.backend.clear_session()

            # Set environment variable
            os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

            gpus = tf.config.list_physical_devices('GPU')
            if gpus:
                # Konfigurasi GPU
                tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
                # Enable mixed precision
                tf.keras.mixed_precision.set_global_policy('mixed_float16')
                print(f"GPU tersedia: {len(gpus)}")
                print("Mixed precision training enabled")
            else:
                print("No GPU found, using CPU")

        except Exception as e:
            print(f"GPU error: {e}")
            print("Continuing with default configuration...")

    @staticmethod
    def create_timestamp_dir(base_dir, prefix='experiment'):
        """Membuat direktori dengan timestamp"""
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        directory = os.path.join(base_dir, f'{prefix}_{timestamp}')
        os.makedirs(directory, exist_ok=True)
        return directory, timestamp

    @staticmethod
    def preprocess_image(image_path, target_size=(224, 224)):
        """Preprocessing gambar untuk inferensi"""
        try:
            # Baca gambar
            img = cv2.imread(image_path)
            if img is None:
                raise ValueError(f"Could not read image: {image_path}")

            # Convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Resize
            img = cv2.resize(img, target_size)

            # Normalisasi
            img = img.astype(np.float32) / 255.0

            # Standarisasi
            mean = np.mean(img, axis=(0,1))
            std = np.std(img, axis=(0,1))
            img = (img - mean) / (std + 1e-7)

            return img

        except Exception as e:
            print(f"Error preprocessing image {image_path}: {str(e)}")
            return None

    @staticmethod
    def calculate_advanced_metrics(y_true, y_pred, y_pred_proba, class_names):
        """Menghitung metrik evaluasi lanjutan"""
        metrics = {}

        # ROC AUC untuk setiap kelas
        for i, class_name in enumerate(class_names):
            try:
                metrics[f'auc_{class_name}'] = roc_auc_score(
                    (y_true == i).astype(int),
                    y_pred_proba[:, i]
                )
            except Exception as e:
                print(f"Error calculating AUC for {class_name}: {str(e)}")
                metrics[f'auc_{class_name}'] = None

        # Precision-Recall metrics
        for i, class_name in enumerate(class_names):
            try:
                precision, recall, _ = precision_recall_curve(
                    (y_true == i).astype(int),
                    y_pred_proba[:, i]
                )
                metrics[f'avg_precision_{class_name}'] = np.mean(precision)
                metrics[f'avg_recall_{class_name}'] = np.mean(recall)
            except Exception as e:
                print(f"Error calculating PR metrics for {class_name}: {str(e)}")
                metrics[f'avg_precision_{class_name}'] = None
                metrics[f'avg_recall_{class_name}'] = None

        return metrics

    @staticmethod
    def create_lr_schedule(initial_lr, decay_steps, warmup_steps=0):
        """Membuat learning rate schedule dengan warmup"""
        warmup_lr = initial_lr * 0.1

        def lr_schedule(step):
            # Warmup phase
            if step < warmup_steps:
                return warmup_lr + (initial_lr - warmup_lr) * (step / warmup_steps)

            # Cosine decay with restarts
            step_after_warmup = step - warmup_steps
            cosine_decay = 0.5 * (1 + np.cos(np.pi * step_after_warmup / decay_steps))
            decayed_lr = initial_lr * cosine_decay

            return decayed_lr

        return lr_schedule

    @staticmethod
    def save_model_summary(model, filepath):
        """Menyimpan ringkasan arsitektur model ke file"""
        try:
            with open(filepath, 'w') as f:
                model.summary(print_fn=lambda x: f.write(x + '\\n'))
            print(f"Model summary saved to: {filepath}")
        except Exception as e:
            print(f"Error saving model summary: {str(e)}")

    @staticmethod
    def get_model_size(model):
        """Menghitung ukuran model dalam MB"""
        try:
            size_bytes = 0
            for weight in model.weights:
                size_bytes += np.prod(weight.shape) * weight.dtype.size
            return size_bytes / (1024 * 1024)  # Convert to MB
        except Exception as e:
            print(f"Error calculating model size: {str(e)}")
            return None
''')

In [18]:
# Pertama, buat script untuk menulis file trainer.py
with open('/content/kanker-kulit-projek/src/trainer.py', 'w') as f:
    f.write('''import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import tensorflow as tf
from datetime import datetime

class EnhancedModelTrainer:
    def __init__(self, config, model):
        self.config = config
        self.model = model
        self.history = None
        self.training_time = None

        # Pastikan semua direktori yang diperlukan ada
        os.makedirs(self.config.CHECKPOINT_DIR, exist_ok=True)
        os.makedirs(self.config.RESULTS_DIR, exist_ok=True)
        os.makedirs(os.path.join(self.config.RESULTS_DIR, 'logs'), exist_ok=True)

        # Setup timestamp untuk logging
        self.timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

    def create_callbacks(self):
        return [
            ModelCheckpoint(
                os.path.join(self.config.CHECKPOINT_DIR, 'best_model_acc.keras'),
                monitor='val_accuracy',
                mode='max',
                save_best_only=True,
                verbose=1
            ),
            ModelCheckpoint(
                os.path.join(self.config.CHECKPOINT_DIR, 'best_model_loss.keras'),
                monitor='val_loss',
                mode='min',
                save_best_only=True,
                verbose=1
            ),
            EarlyStopping(
                monitor='val_loss',
                patience=20,
                restore_best_weights=True,
                verbose=1
            ),
            TensorBoard(
                log_dir=os.path.join(self.config.RESULTS_DIR, 'logs', self.timestamp),
                histogram_freq=1,
                write_graph=True,
                write_images=True,
                update_freq='epoch',
                profile_batch=0
            ),
            CSVLogger(
                os.path.join(self.config.RESULTS_DIR, f'training_log_{self.timestamp}.csv'),
                separator=',',
                append=False
            )
        ]

    def train(self, train_generator, val_generator):
        print("\\nMemulai proses pelatihan model kanker kulit...")
        print(f"Timestamp: {self.timestamp}")

        try:
            start_time = time.time()

            # Enable mixed precision
            if tf.config.list_physical_devices('GPU'):
                try:
                    policy = tf.keras.mixed_precision.Policy('mixed_float16')
                    tf.keras.mixed_precision.set_global_policy(policy)
                    print("Mixed precision training enabled pada GPU")
                except Exception as e:
                    print(f"Warning: Could not enable mixed precision: {str(e)}")

            # Training
            self.history = self.model.fit(
                train_generator,
                epochs=self.config.EPOCHS,
                validation_data=val_generator,
                callbacks=self.create_callbacks(),
                verbose=1
            )

            # Catat waktu training
            self.training_time = time.time() - start_time
            hours = int(self.training_time // 3600)
            minutes = int((self.training_time % 3600) // 60)
            seconds = int(self.training_time % 60)

            print(f"\\nPelatihan selesai dalam: {hours}h {minutes}m {seconds}s")

            return self.history

        except KeyboardInterrupt:
            print("\\nPelatihan dihentikan oleh user")
            return self.history
        except Exception as e:
            print(f"\\nError selama pelatihan: {str(e)}")
            raise e

    def evaluate(self, test_generator):
        print("\\nMemulai evaluasi model pada data testing...")

        # Evaluasi
        metrics = self.model.evaluate(
            test_generator,
            verbose=1,
            return_dict=True
        )

        # Prediksi
        predictions = []
        true_labels = []
        pred_probs = []

        print("\\nMengumpulkan prediksi...")
        for i in range(len(test_generator)):
            x, y = next(test_generator)
            pred = self.model.predict(x, verbose=0)
            predictions.extend(np.argmax(pred, axis=1))
            true_labels.extend(np.argmax(y, axis=1))
            pred_probs.extend(pred)

            if len(predictions) >= len(test_generator.labels):
                break

        y_pred = np.array(predictions)
        y_true = np.array(true_labels)
        y_pred_probs = np.array(pred_probs)

        # Print semua metrics
        print(f"\\nMetrik Evaluasi:")
        for metric_name, metric_value in metrics.items():
            print(f"{metric_name}: {metric_value:.4f}")

        # Classification report
        print("\\nLaporan Klasifikasi Detail:")
        class_names = list(self.config.CLASS_MAPPING.values())
        report = classification_report(
            y_true, y_pred,
            target_names=class_names,
            digits=4
        )
        print(report)

        # Simpan report ke file
        with open(os.path.join(self.config.RESULTS_DIR, f'classification_report_{self.timestamp}.txt'), 'w') as f:
            f.write("HASIL EVALUASI MODEL KLASIFIKASI KANKER KULIT\\n")
            f.write(f"Timestamp: {self.timestamp}\\n\\n")
            f.write("Metrik Evaluasi:\\n")
            for metric_name, metric_value in metrics.items():
                f.write(f"{metric_name}: {metric_value:.4f}\\n")
            f.write("\\nLaporan Klasifikasi Detail:\\n")
            f.write(report)
            if self.training_time:
                f.write(f"\\nTotal waktu training: {self.training_time:.2f} detik")

        # Plot visualisasi
        self._plot_confusion_matrix(y_true, y_pred, class_names)
        self._plot_roc_curves(y_true, y_pred_probs, class_names)
        if self.history:
            self._plot_training_history()

    def _plot_confusion_matrix(self, y_true, y_pred, class_names):
        plt.figure(figsize=(12, 10))
        cm = confusion_matrix(y_true, y_pred)

        # Normalisasi
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Plot heatmap
        sns.heatmap(
            cm_norm,
            annot=cm,
            fmt='d',
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names
        )

        plt.title('Matriks Konfusi Klasifikasi Kanker Kulit')
        plt.ylabel('Label Sebenarnya')
        plt.xlabel('Label Prediksi')
        plt.tight_layout()

        # Simpan plot
        confusion_matrix_path = os.path.join(self.config.RESULTS_DIR, f'confusion_matrix_{self.timestamp}.png')
        plt.savefig(confusion_matrix_path, dpi=300, bbox_inches='tight')
        plt.close()

    def _plot_roc_curves(self, y_true, y_pred_probs, class_names):
        plt.figure(figsize=(10, 8))

        # One-hot encode true labels
        y_true_oh = tf.keras.utils.to_categorical(y_true)

        # Plot ROC curve untuk setiap kelas
        for i in range(len(class_names)):
            fpr, tpr, _ = roc_curve(y_true_oh[:, i], y_pred_probs[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves per Kelas')
        plt.legend(loc="lower right")
        plt.grid(True)

        # Simpan plot
        roc_path = os.path.join(self.config.RESULTS_DIR, f'roc_curves_{self.timestamp}.png')
        plt.savefig(roc_path, dpi=300, bbox_inches='tight')
        plt.close()

    def _plot_training_history(self):
        plt.figure(figsize=(15, 5))

        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(self.history.history['accuracy'], label='Training')
        plt.plot(self.history.history['val_accuracy'], label='Validasi')
        plt.title('Akurasi Model per Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Akurasi')
        plt.legend()
        plt.grid(True)

        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(self.history.history['loss'], label='Training')
        plt.plot(self.history.history['val_loss'], label='Validasi')
        plt.title('Loss Model per Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()

        # Simpan plot
        history_path = os.path.join(self.config.RESULTS_DIR, f'training_history_{self.timestamp}.png')
        plt.savefig(history_path, dpi=300, bbox_inches='tight')
        plt.close()
''')

In [34]:
with open('/content/kanker-kulit-projek/src/main.py', 'w') as f:
    f.write('''import tensorflow as tf
import os
import time
import logging
import traceback
from datetime import datetime
from config import Config
from data_processor import DataProcessor
from model import ModelBuilder
from trainer import EnhancedModelTrainer

def setup_logging(config):
    """Setup logging configuration"""
    log_dir = os.path.join(config.get_experiment_dir(), 'logs')
    os.makedirs(log_dir, exist_ok=True)

    log_format = '%(asctime)s - %(levelname)s - %(message)s'
    logging.basicConfig(
        level=logging.INFO,
        format=log_format,
        handlers=[
            logging.FileHandler(
                os.path.join(log_dir, f'training_{config.timestamp}.log')
            ),
            logging.StreamHandler()
        ]
    )

def configure_gpu():
    """Konfigurasi GPU sebelum memulai"""
    try:
        # Set environment variable
        os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

        # Dapatkan daftar GPU
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            # Konfigurasi GPU
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            # Enable mixed precision
            tf.keras.mixed_precision.set_global_policy('mixed_float16')
    except Exception as e:
        logging.error(f"Error configuring GPU: {str(e)}")
        logging.info("Continuing with default configuration...")

def print_system_info():
    """Print system and environment information"""
    logging.info("System Information:")
    logging.info(f"TensorFlow version: {tf.__version__}")
    logging.info(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
    logging.info(f"Eager Execution: {tf.executing_eagerly()}")

def main():
    """Program utama klasifikasi kanker kulit"""
    try:
        start_time = time.time()

        # Konfigurasi GPU dulu sebelum melakukan operasi lain
        configure_gpu()

        # Initialize configuration
        config = Config()

        # Setup logging
        setup_logging(config)

        # Print header
        logging.info("="*50)
        logging.info("PROGRAM KLASIFIKASI KANKER KULIT")
        logging.info(f"Timestamp: {config.timestamp}")
        logging.info("="*50)

        # Print system information
        print_system_info()

        # Set random seeds for reproducibility
        tf.random.set_seed(config.RANDOM_SEED)

        # Initialize data processor
        logging.info("\\nMempersiapkan data...")
        data_processor = DataProcessor(config)

        try:
            # Prepare data menggunakan method yang sudah ada
            train_generator, val_generator, test_generator = data_processor.prepare_data()

            # Build and compile model
            logging.info("\\nMembangun model...")
            model_builder = ModelBuilder(config)
            model = model_builder.build()  # Menggunakan method build() bukan build_model()

            # Initialize trainer
            trainer = EnhancedModelTrainer(config, model)

            # Train model with progress tracking
            logging.info("\\nMemulai proses training...")
            history = trainer.train(train_generator, val_generator)

            # Evaluate model
            logging.info("\\nMemulai proses evaluasi...")
            trainer.evaluate(test_generator)

            # Calculate total execution time
            end_time = time.time()
            hours, rem = divmod(end_time - start_time, 3600)
            minutes, seconds = divmod(rem, 60)

            logging.info("\\nProgram selesai!")
            logging.info(f"Total waktu eksekusi: {int(hours)}h {int(minutes)}m {int(seconds)}s")
            logging.info(f"Hasil visualisasi tersimpan di: {config.RESULTS_DIR}")
            logging.info(f"Model terbaik tersimpan di: {config.CHECKPOINT_DIR}")
            logging.info(f"Log lengkap tersimpan di: {config.get_experiment_dir()}/logs")

        except KeyboardInterrupt:
            logging.warning("\\nProgram dihentikan oleh user")

        except Exception as e:
            logging.error(f"\\nError selama eksekusi: {str(e)}")
            logging.error(traceback.format_exc())
            raise e

    except Exception as e:
        logging.error(f"Fatal error: {str(e)}")
        logging.error(traceback.format_exc())
        raise e

if __name__ == "__main__":
    main()
''')

In [39]:
%cd /content/kanker-kulit-projek
!python src/main.py

/content/kanker-kulit-projek
2025-02-11 18:07:16.306048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739297236.324675    9222 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739297236.330253    9222 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 18:07:22,222 - INFO - ==================================================
2025-02-11 18:07:22,222 - INFO - PROGRAM KLASIFIKASI KANKER KULIT
2025-02-11 18:07:22,223 - INFO - Timestamp: 20250211-180722
2025-02-11 18:07:22,223 - INFO - ==================================================
2025-02-11 18:07:22,223 - INFO - System Information:
2025-02-11 18:07:22,223 - INFO - TensorFlow version: 2.18.0
2